In [29]:
# Import Modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Display all of the columns
pd.set_option('display.max_columns', None)

In [30]:
# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(
    Path('data/board_games_complete.csv')
)
df.head()

,rank,bgg_url,id,name,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,yearpublished,average_rating,geek_rating,users_rated,image_url,thumb_url,age,mechanic,total_owners,category,designer,publisher,total_weights,expands,reimplements,num_fans,mechanic_count,mechanic_clean,category_count,category_clean,designer_count,designer_clean,name_clean,type,minage,bayes_average_rating,total_traders,total_wanters,total_wishers,total_comments,average_weight
0,1,https://boardgamegeek.com/boardgame/174430/glo...,174430,Gloomhaven,1,4,120,60,120,2017,8.91603,8.61793,22367,https://cf.geekdo-images.com/original/img/lDN3...,https://cf.geekdo-images.com/thumb/img/e7GyV4P...,12,"Campaign / Battle Card Driven, Cooperative Pla...",36778,"Adventure, Exploration, Fantasy, Fighting, Min...",Isaac Childres,Cephalofair Games,3.7768,NaN,NaN,4413,9,Campaign / Battle Card Driven,5,Adventure,1,Isaac Childres,Gloomhaven,boardgame,12.0,0.00000,0.0,62.0,476.0,39.0,3.2000
1,2,https://boardgamegeek.com/boardgame/161936/pan...,161936,Pandemic Legacy: Season 1,2,4,60,60,60,2015,8.65105,8.49375,29726,https://cf.geekdo-images.com/original/img/P_Sw...,https://cf.geekdo-images.com/thumb/img/WI5NmPd...,13,"Action Point Allowance System, Cooperative Pla...",47498,"Environmental, Medical","Rob Daviau, Matt Leacock","Z-Man Games, Inc.",2.8323,NaN,30549.0,2168,7,Action Point Allowance System,2,Environmental,2,Rob Daviau,Pandemic Legacy Season,boardgame,13.0,0.00000,0.0,277.0,1853.0,50.0,3.0000
2,3,https://boardgamegeek.com/boardgame/182028/thr...,182028,Through the Ages: A New Story of Civilization,2,4,120,120,120,2015,8.55251,8.27103,14790,https://cf.geekdo-images.com/original/img/1d2h...,https://cf.geekdo-images.com/thumb/img/Ohqc2KT...,14,"Action Point Allowance System, Auction/Bidding...",18489,"Card Game, Civilization, Economic",Vlaada Chvátil,Czech Games Edition,4.3715,NaN,25613.0,1054,3,Action Point Allowance System,3,Card Game,1,Vlaada Chvátil,Through the Ages A New Story of Civilization,boardgame,14.0,0.00000,0.0,84.0,686.0,23.0,4.2000
3,4,https://boardgamegeek.com/boardgame/167791/ter...,167791,Terraforming Mars,1,5,120,120,120,2016,8.39916,8.23172,35253,https://cf.geekdo-images.com/original/img/o8z_...,https://cf.geekdo-images.com/thumb/img/yFqQ569...,12,"Card Drafting, Hand Management, Set Collection...",45204,"Economic, Environmental, Industry / Manufactur...",Jacob Fryxelius,FryxGames,3.2344,NaN,NaN,2718,6,Card Drafting,6,Economic,1,Jacob Fryxelius,Terraforming Mars,boardgame,12.0,0.00000,0.0,37.0,224.0,22.0,3.8333
4,5,https://boardgamegeek.com/boardgame/12333/twil...,12333,Twilight Struggle,2,2,180,120,180,2005,8.32843,8.18051,33489,https://cf.geekdo-images.com/original/img/ZPnn...,https://cf.geekdo-images.com/thumb/img/mEmeJrI...,13,"Area Control / Area Influence, Campaign / Batt...",45955,"Modern Warfare, Political, Wargame","Ananda Gupta, Jason Matthews",GMT Games,3.5627,NaN,NaN,3639,5,Area Control / Area Influence,3,Modern Warfare,2,Ananda Gupta,Twilight Struggle,boardgame,13.0,8.22186,372.0,1219.0,5865.0,5347.0,3.4785


In [31]:
# Process data
df['category_clean'] = df['category_clean'].replace({'Animals': 'Other','Card Game': 'Other', 'Dice': 'Other','Electronic': 'Other','Game System': 'Other','Horror': 'Other','Humor': 'Other','Mature / Adult': 'Other','Miniatures': 'Other','Music': 'Other','none': 'Other','Party Game': 'Other','Religious': 'Other','Fighting': 'Other','Racing': 'Other','Sports': 'Other','Age of Reason': 'Era','American West': 'Era','Ancient': 'Era','Medieval': 'Era','Napoleonic': 'Era','Post-Napoleonic': 'Era','Prehistoric': 'Era','Renaissance': 'Era','Abstract Strategy': 'Strategy','Bluffing': 'Strategy','City Building': 'Strategy','Civilization': 'Strategy','Collectible Components': 'Strategy','Deduction': 'Strategy','Economic': 'Strategy','Exploration': 'Strategy','Farming': 'Strategy','Industry / Manufacturing': 'Strategy','Mafia': 'Strategy','Maze': 'Strategy','Murder/Mystery': 'Strategy','Negotiation': 'Strategy','Pirates': 'Strategy','Political': 'Strategy','Puzzle': 'Strategy','Real-time': 'Strategy','Spies/Secret Agents': 'Strategy','Territory Building': 'Strategy','Trains': 'Strategy','Transportation': 'Strategy','Zombies': 'Strategy','Action / Dexterity': 'Adventure','Adventure': 'Adventure','Arabian': 'Adventure','Aviation / Flight': 'Adventure','Comic Book / Strip': 'Adventure','Fantasy': 'Adventure','Movies / TV / Radio theme': 'Adventure','Mythology': 'Adventure','Nautical': 'Adventure','Pike and Shot': 'Adventure','Print & Play': 'Adventure','Science Fiction': 'Adventure','Space Exploration': 'Adventure','Travel': 'Adventure','Video Game Theme': 'Adventure','Book': 'Educational',"Children's Game": 'Educational','Educational': 'Educational','Environmental': 'Educational','Math': 'Educational','Medical': 'Educational','Memory': 'Educational','Novel-based': 'Educational','Number': 'Educational','Trivia': 'Educational','Word Game': 'Educational','American Civil War': 'Wargame','American Indian Wars': 'Wargame','American Revolutionary War': 'Wargame','Civil War': 'Wargame','Korean War': 'Wargame','Modern Warfare': 'Wargame','Vietnam War': 'Wargame','Wargame':'Wargame','World War I':'Wargame','World War II':'Wargame',})

In [32]:
# Process data
df['mechanic_clean'] = df['mechanic_clean'].replace({'Cooperative Play': 'Co-op',
'Dice Rolling': 'Dice_Rolling',
'Roll / Spin and Move': 'Dice_Rolling',
'Simultaneous Action Selection': 'PVP',
'Chit-Pull System': 'PVP',
'Campaign / Battle Card Driven': 'PVP',
'Hex-and-Counter': 'PVP',
'Area Control / Area Influence': 'PVP',
'Action / Movement Programming': 'PVP',
'Variable Player Powers': 'PVP',
'Betting/Wagering': 'PVP',
'Trick-taking': 'PVP',
'Player Elimination': 'PVP',
'Take That': 'PVP',
'Secret Unit Deployment': 'PVP',
'Rock-Paper-Scissors': 'PVP',
'Area Movement': 'Movement',
'Grid Movement': 'Movement',
'Modular Board': 'Movement',
'Tile Placement': 'Movement',
'Line Drawing': 'Movement',
'Pattern Recognition': 'Movement',
'Area Enclosure': 'Movement',
'Route/Network Building': 'Movement',
'Pick-up and Deliver': 'Movement',
'Crayon Rail System': 'Movement',
'Point to Point Movement': 'Movement',
'Time Track': 'Movement',
'none': 'Other',
'Action Point Allowance System': 'Other',
'Hand Management': 'Other',
'Worker Placement': 'Other',
'Pattern Building': 'Other',
'Auction/Bidding': 'Other',
'Card Drafting': 'Other',
'Paper-and-Pencil': 'Other',
'Press Your Luck': 'Other',
'Deck / Pool Building': 'Other',
'Set Collection': 'Other',
'Partnerships': 'Other',
'Commodity Speculation': 'Other',
'Area-Impulse': 'Other',
'Voting': 'Other',
'Stock Holding': 'Other',
'Memory': 'Other',
'Variable Phase Order': 'Other',
'Trading': 'Other',
'Storytelling': 'Role_Playing',
'Simulation': 'Role_Playing',
'Role Playing': 'Role_Playing',
'Acting': 'Role_Playing',
'Singing': 'Role_Playing',
})

In [33]:
df.head()

,rank,bgg_url,id,name,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,yearpublished,average_rating,geek_rating,users_rated,image_url,thumb_url,age,mechanic,total_owners,category,designer,publisher,total_weights,expands,reimplements,num_fans,mechanic_count,mechanic_clean,category_count,category_clean,designer_count,designer_clean,name_clean,type,minage,bayes_average_rating,total_traders,total_wanters,total_wishers,total_comments,average_weight
0,1,https://boardgamegeek.com/boardgame/174430/glo...,174430,Gloomhaven,1,4,120,60,120,2017,8.91603,8.61793,22367,https://cf.geekdo-images.com/original/img/lDN3...,https://cf.geekdo-images.com/thumb/img/e7GyV4P...,12,"Campaign / Battle Card Driven, Cooperative Pla...",36778,"Adventure, Exploration, Fantasy, Fighting, Min...",Isaac Childres,Cephalofair Games,3.7768,NaN,NaN,4413,9,PVP,5,Adventure,1,Isaac Childres,Gloomhaven,boardgame,12.0,0.00000,0.0,62.0,476.0,39.0,3.2000
1,2,https://boardgamegeek.com/boardgame/161936/pan...,161936,Pandemic Legacy: Season 1,2,4,60,60,60,2015,8.65105,8.49375,29726,https://cf.geekdo-images.com/original/img/P_Sw...,https://cf.geekdo-images.com/thumb/img/WI5NmPd...,13,"Action Point Allowance System, Cooperative Pla...",47498,"Environmental, Medical","Rob Daviau, Matt Leacock","Z-Man Games, Inc.",2.8323,NaN,30549.0,2168,7,Other,2,Educational,2,Rob Daviau,Pandemic Legacy Season,boardgame,13.0,0.00000,0.0,277.0,1853.0,50.0,3.0000
2,3,https://boardgamegeek.com/boardgame/182028/thr...,182028,Through the Ages: A New Story of Civilization,2,4,120,120,120,2015,8.55251,8.27103,14790,https://cf.geekdo-images.com/original/img/1d2h...,https://cf.geekdo-images.com/thumb/img/Ohqc2KT...,14,"Action Point Allowance System, Auction/Bidding...",18489,"Card Game, Civilization, Economic",Vlaada Chvátil,Czech Games Edition,4.3715,NaN,25613.0,1054,3,Other,3,Other,1,Vlaada Chvátil,Through the Ages A New Story of Civilization,boardgame,14.0,0.00000,0.0,84.0,686.0,23.0,4.2000
3,4,https://boardgamegeek.com/boardgame/167791/ter...,167791,Terraforming Mars,1,5,120,120,120,2016,8.39916,8.23172,35253,https://cf.geekdo-images.com/original/img/o8z_...,https://cf.geekdo-images.com/thumb/img/yFqQ569...,12,"Card Drafting, Hand Management, Set Collection...",45204,"Economic, Environmental, Industry / Manufactur...",Jacob Fryxelius,FryxGames,3.2344,NaN,NaN,2718,6,Other,6,Strategy,1,Jacob Fryxelius,Terraforming Mars,boardgame,12.0,0.00000,0.0,37.0,224.0,22.0,3.8333
4,5,https://boardgamegeek.com/boardgame/12333/twil...,12333,Twilight Struggle,2,2,180,120,180,2005,8.32843,8.18051,33489,https://cf.geekdo-images.com/original/img/ZPnn...,https://cf.geekdo-images.com/thumb/img/mEmeJrI...,13,"Area Control / Area Influence, Campaign / Batt...",45955,"Modern Warfare, Political, Wargame","Ananda Gupta, Jason Matthews",GMT Games,3.5627,NaN,NaN,3639,5,PVP,3,Wargame,2,Ananda Gupta,Twilight Struggle,boardgame,13.0,8.22186,372.0,1219.0,5865.0,5347.0,3.4785


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   rank                  5000 non-null   int64  
 1   bgg_url               5000 non-null   object 
 2   id                    5000 non-null   int64  
 3   name                  5000 non-null   object 
 4   minplayers            5000 non-null   int64  
 5   maxplayers            5000 non-null   int64  
 6   playingtime           5000 non-null   int64  
 7   minplaytime           5000 non-null   int64  
 8   maxplaytime           5000 non-null   int64  
 9   yearpublished         5000 non-null   int64  
 10  average_rating        5000 non-null   float64
 11  geek_rating           5000 non-null   float64
 12  users_rated           5000 non-null   int64  
 13  image_url             5000 non-null   object 
 14  thumb_url             5000 non-null   object 
 15  age                  

In [35]:
# drop unwanted columns
df = df.drop(['bgg_url',
'yearpublished',
'geek_rating',
'users_rated',
'image_url',
'thumb_url',
'mechanic',
'category',
'designer',
'publisher',
'expands',
'reimplements',
'type',
'minage',
'bayes_average_rating',
'total_traders',
'total_wanters',
'total_wishers',
'total_comments',
'average_weight',
'name',
'playingtime',
'minplaytime',
'num_fans',
'designer_clean',
], axis=1)


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   rank            5000 non-null   int64  
 1   id              5000 non-null   int64  
 2   minplayers      5000 non-null   int64  
 3   maxplayers      5000 non-null   int64  
 4   maxplaytime     5000 non-null   int64  
 5   average_rating  5000 non-null   float64
 6   age             5000 non-null   int64  
 7   total_owners    5000 non-null   int64  
 8   total_weights   5000 non-null   float64
 9   mechanic_count  5000 non-null   int64  
 10  mechanic_clean  5000 non-null   object 
 11  category_count  5000 non-null   int64  
 12  category_clean  5000 non-null   object 
 13  designer_count  5000 non-null   int64  
 14  name_clean      4988 non-null   object 
dtypes: float64(2), int64(10), object(3)
memory usage: 586.1+ KB


In [37]:
df = df.dropna()

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4988 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   rank            4988 non-null   int64  
 1   id              4988 non-null   int64  
 2   minplayers      4988 non-null   int64  
 3   maxplayers      4988 non-null   int64  
 4   maxplaytime     4988 non-null   int64  
 5   average_rating  4988 non-null   float64
 6   age             4988 non-null   int64  
 7   total_owners    4988 non-null   int64  
 8   total_weights   4988 non-null   float64
 9   mechanic_count  4988 non-null   int64  
 10  mechanic_clean  4988 non-null   object 
 11  category_count  4988 non-null   int64  
 12  category_clean  4988 non-null   object 
 13  designer_count  4988 non-null   int64  
 14  name_clean      4988 non-null   object 
dtypes: float64(2), int64(10), object(3)
memory usage: 623.5+ KB


In [39]:
ohe_df = pd.get_dummies(df, 
                        columns = ['category_clean', 'mechanic_clean'], 
                        prefix = ['category_clean', 'mechanic_clean',])
ohe_df['category_clean'] = df.category_clean
ohe_df['mechanic_clean'] = df.mechanic_clean
ohe_df.head()

,rank,id,minplayers,maxplayers,maxplaytime,average_rating,age,total_owners,total_weights,mechanic_count,category_count,designer_count,name_clean,category_clean_Adventure,category_clean_Educational,category_clean_Era,category_clean_Expansion for Base-game,category_clean_Other,category_clean_Strategy,category_clean_Wargame,mechanic_clean_Co-op,mechanic_clean_Dice_Rolling,mechanic_clean_Movement,mechanic_clean_Other,mechanic_clean_PVP,mechanic_clean_Role_Playing,category_clean,mechanic_clean
0,1,174430,1,4,120,8.91603,12,36778,3.7768,9,5,1,Gloomhaven,1,0,0,0,0,0,0,0,0,0,0,1,0,Adventure,PVP
1,2,161936,2,4,60,8.65105,13,47498,2.8323,7,2,2,Pandemic Legacy Season,0,1,0,0,0,0,0,0,0,0,1,0,0,Educational,Other
2,3,182028,2,4,120,8.55251,14,18489,4.3715,3,3,1,Through the Ages A New Story of Civilization,0,0,0,0,1,0,0,0,0,0,1,0,0,Other,Other
3,4,167791,1,5,120,8.39916,12,45204,3.2344,6,6,1,Terraforming Mars,0,0,0,0,0,1,0,0,0,0,1,0,0,Strategy,Other
4,5,12333,2,2,180,8.32843,13,45955,3.5627,5,3,2,Twilight Struggle,0,0,0,0,0,0,1,0,0,0,0,1,0,Wargame,PVP


In [40]:
ohe_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4988 entries, 0 to 4999
Data columns (total 28 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   rank                                    4988 non-null   int64  
 1   id                                      4988 non-null   int64  
 2   minplayers                              4988 non-null   int64  
 3   maxplayers                              4988 non-null   int64  
 4   maxplaytime                             4988 non-null   int64  
 5   average_rating                          4988 non-null   float64
 6   age                                     4988 non-null   int64  
 7   total_owners                            4988 non-null   int64  
 8   total_weights                           4988 non-null   float64
 9   mechanic_count                          4988 non-null   int64  
 10  category_count                          4988 non-null   int6

In [41]:
ohe_df.to_csv("boardgames.csv", index=False)

In [42]:
ohe_df.columns

Index(['rank', 'id', 'minplayers', 'maxplayers', 'maxplaytime',
       'average_rating', 'age', 'total_owners', 'total_weights',
       'mechanic_count', 'category_count', 'designer_count', 'name_clean',
       'category_clean_Adventure', 'category_clean_Educational',
       'category_clean_Era', 'category_clean_Expansion for Base-game',
       'category_clean_Other', 'category_clean_Strategy',
       'category_clean_Wargame', 'mechanic_clean_Co-op',
       'mechanic_clean_Dice_Rolling', 'mechanic_clean_Movement',
       'mechanic_clean_Other', 'mechanic_clean_PVP',
       'mechanic_clean_Role_Playing', 'category_clean', 'mechanic_clean'],
      dtype='object')

In [43]:
 # select features for the model
features = ['minplayers', 'maxplayers', 'maxplaytime',
        'age', 'total_weights',
       'mechanic_count', 'category_count', 'designer_count', 
       'category_clean_Adventure', 'category_clean_Educational',
       'category_clean_Era', 'category_clean_Expansion for Base-game',
       'category_clean_Other', 'category_clean_Strategy',
       'category_clean_Wargame', 'mechanic_clean_Co-op',
       'mechanic_clean_Dice_Rolling', 'mechanic_clean_Movement',
       'mechanic_clean_Other', 'mechanic_clean_PVP',
       'mechanic_clean_Role_Playing']
# create a dataframe with only the relevant features
df_model = ohe_df.loc[:, ['id', 'name_clean','category_clean', 'mechanic_clean', 'average_rating', 'total_owners'] + features]
#df = df[['id', 'name_clean', 'category_clean', 'mechanic_clean', 'total_weights', 'average_rating', 'age', 'maxplaytime', 'minplayers', 'maxplayers']]
df_model.head()

,id,name_clean,category_clean,mechanic_clean,average_rating,total_owners,minplayers,maxplayers,maxplaytime,age,total_weights,mechanic_count,category_count,designer_count,category_clean_Adventure,category_clean_Educational,category_clean_Era,category_clean_Expansion for Base-game,category_clean_Other,category_clean_Strategy,category_clean_Wargame,mechanic_clean_Co-op,mechanic_clean_Dice_Rolling,mechanic_clean_Movement,mechanic_clean_Other,mechanic_clean_PVP,mechanic_clean_Role_Playing
0,174430,Gloomhaven,Adventure,PVP,8.91603,36778,1,4,120,12,3.7768,9,5,1,1,0,0,0,0,0,0,0,0,0,0,1,0
1,161936,Pandemic Legacy Season,Educational,Other,8.65105,47498,2,4,60,13,2.8323,7,2,2,0,1,0,0,0,0,0,0,0,0,1,0,0
2,182028,Through the Ages A New Story of Civilization,Other,Other,8.55251,18489,2,4,120,14,4.3715,3,3,1,0,0,0,0,1,0,0,0,0,0,1,0,0
3,167791,Terraforming Mars,Strategy,Other,8.39916,45204,1,5,120,12,3.2344,6,6,1,0,0,0,0,0,1,0,0,0,0,1,0,0
4,12333,Twilight Struggle,Wargame,PVP,8.32843,45955,2,2,180,13,3.5627,5,3,2,0,0,0,0,0,0,1,0,0,0,0,1,0


In [44]:
from sklearn.neighbors import NearestNeighbors

# define the number of nearest neighbors to consider
k = 10

# initialize the model with the number of neighbors
model = NearestNeighbors(n_neighbors=k)

# fit the model to the data
model.fit(df_model[features])

# define a function to recommend beers based on a given beer name
def recommend_boardgame(boardgame_name, df_model, features):
    # get the beer_id of the given beer name
    boardgame_id = df_model[df_model['name_clean'] == boardgame_name]['id'].iloc[0]
    
    # get the index of the beer in the model dataframe
    idx = df_model[df_model['id'] == boardgame_id].index[0]
    
    # get the features of the beer
    boardgame_features = df_model.loc[idx, features].values.reshape(1, -1)
    
    # find the k nearest neighbors
    distances, indices = model.kneighbors(boardgame_features)
    
    # get the beer names of the nearest neighbors
    boardgames = df_model.iloc[indices[0]]
    boardgames["distance"]=distances[0]
    
    return boardgames

In [45]:
recommend_boardgame('Catan', df_model, features)

C:\Users\vneis\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\vneis\AppData\Local\Temp\ipykernel_25004\2259701537.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boardgames["distance"]=distances[0]


,id,name_clean,category_clean,mechanic_clean,average_rating,total_owners,minplayers,maxplayers,maxplaytime,age,total_weights,mechanic_count,category_count,designer_count,category_clean_Adventure,category_clean_Educational,category_clean_Era,category_clean_Expansion for Base-game,category_clean_Other,category_clean_Strategy,category_clean_Wargame,mechanic_clean_Co-op,mechanic_clean_Dice_Rolling,mechanic_clean_Movement,mechanic_clean_Other,mechanic_clean_PVP,mechanic_clean_Role_Playing,distance
311,13,Catan,Strategy,Dice_Rolling,7.20035,121071,3,4,120,10,2.3423,4,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0.000000
3015,7571,Winds of Plunder,Strategy,Other,6.44560,946,3,5,120,10,2.3628,4,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1.732172
1124,1568,Space Crusade,Other,Dice_Rolling,6.97261,3923,2,4,120,10,2.2933,4,2,1,0,0,0,0,1,0,0,0,1,0,0,0,0,2.000600
828,202670,Ticket to Ride Rails Sails,Strategy,Other,7.40204,3764,2,5,120,10,2.6857,4,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,2.029267
3719,59936,Battle Merchants,Strategy,Other,6.82306,544,2,4,120,10,2.8824,4,2,1,0,0,0,0,0,1,0,0,0,0,1,0,0,2.071644
3068,103091,Catan Histories Merchants of Europe,Strategy,Dice_Rolling,6.75782,1343,3,4,120,12,2.4146,3,2,1,0,0,0,0,0,1,0,0,1,0,0,0,0,2.450557
4794,87120,Vintage,Educational,Movement,6.57772,332,2,4,120,10,2.9545,4,2,1,0,1,0,0,0,0,0,0,0,1,0,0,0,2.524834
442,30380,Cuba,Strategy,Other,7.15845,7711,2,5,120,10,3.2698,4,2,2,0,0,0,0,0,1,0,0,0,0,1,0,0,2.619209
1826,1621,Mutant Chronicles Siege of the Citadel,Adventure,Other,7.00912,1621,2,5,120,10,2.2333,4,2,1,1,0,0,0,0,0,0,0,0,0,1,0,0,2.647996
98,182874,Grand Austria Hotel,Strategy,Dice_Rolling,7.78280,9676,2,4,120,12,3.2273,3,1,2,0,0,0,0,0,1,0,0,1,0,0,0,0,2.789843


In [46]:
# test scaled data
df_scaled = ohe_df.copy() 

In [48]:
 # select features for the model
features = ['minplayers', 'maxplayers', 'maxplaytime',
        'age', 'total_weights',
       'mechanic_count', 'category_count', 'designer_count', 
       'category_clean_Adventure', 'category_clean_Educational',
       'category_clean_Era', 'category_clean_Expansion for Base-game',
       'category_clean_Other', 'category_clean_Strategy',
       'category_clean_Wargame', 'mechanic_clean_Co-op',
       'mechanic_clean_Dice_Rolling', 'mechanic_clean_Movement',
       'mechanic_clean_Other', 'mechanic_clean_PVP',
       'mechanic_clean_Role_Playing']

# scale the features to a range between 0 and 1
scaler = MinMaxScaler()
df_scaled[features] = scaler.fit_transform(df_scaled[features])

# create a dataframe with only the relevant features
df_model = df_scaled.loc[:, ['id', 'name_clean','category_clean', 'mechanic_clean', 'average_rating', 'total_owners'] + features]
#df = df[['id', 'name_clean', 'category_clean', 'mechanic_clean', 'total_weights', 'average_rating', 'age', 'maxplaytime', 'minplayers', 'maxplayers']]
df_model.head()

,id,name_clean,category_clean,mechanic_clean,average_rating,total_owners,minplayers,maxplayers,maxplaytime,age,total_weights,mechanic_count,category_count,designer_count,category_clean_Adventure,category_clean_Educational,category_clean_Era,category_clean_Expansion for Base-game,category_clean_Other,category_clean_Strategy,category_clean_Wargame,mechanic_clean_Co-op,mechanic_clean_Dice_Rolling,mechanic_clean_Movement,mechanic_clean_Other,mechanic_clean_PVP,mechanic_clean_Role_Playing
0,174430,Gloomhaven,Adventure,PVP,8.92,36778,0.125,0.020,0.005333,0.666667,0.774590,0.727273,0.444444,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,161936,Pandemic Legacy Season,Educational,Other,8.65,47498,0.250,0.020,0.002667,0.722222,0.579918,0.545455,0.111111,0.1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,182028,Through the Ages A New Story of Civilization,Other,Other,8.55,18489,0.250,0.020,0.005333,0.777778,0.895492,0.181818,0.222222,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,167791,Terraforming Mars,Strategy,Other,8.40,45204,0.125,0.025,0.005333,0.666667,0.661885,0.454545,0.555556,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,12333,Twilight Struggle,Wargame,PVP,8.33,45955,0.250,0.010,0.008000,0.722222,0.729508,0.363636,0.222222,0.1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [49]:
from sklearn.neighbors import NearestNeighbors

# define the number of nearest neighbors to consider
k = 10

# initialize the model with the number of neighbors
model = NearestNeighbors(n_neighbors=k)

# fit the model to the data
model.fit(df_model[features])

# define a function to recommend boardgames based on a given boardgames name
def recommend_boardgame(boardgame_name, df_model, features):
    # get the beer_id of the given beer name
    boardgame_id = df_model[df_model['name_clean'] == boardgame_name]['id'].iloc[0]
    
    # get the index of the boardgames in the model dataframe
    idx = df_model[df_model['id'] == boardgame_id].index[0]
    
    # get the features of the boardgames
    boardgame_features = df_model.loc[idx, features].values.reshape(1, -1)
    
    # find the k nearest neighbors
    distances, indices = model.kneighbors(boardgame_features)
    
    # get the beer names of the nearest neighbors
    boardgames = df_model.iloc[indices[0]]
    boardgames["distance"] = distances[0]
    
    return boardgames

In [50]:
recommend_boardgame('Catan', df_model, features)

C:\Users\vneis\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\vneis\AppData\Local\Temp\ipykernel_25004\2259701537.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boardgames["distance"]=distances[0]


,id,name_clean,category_clean,mechanic_clean,average_rating,total_owners,minplayers,maxplayers,maxplaytime,age,total_weights,mechanic_count,category_count,designer_count,category_clean_Adventure,category_clean_Educational,category_clean_Era,category_clean_Expansion for Base-game,category_clean_Other,category_clean_Strategy,category_clean_Wargame,mechanic_clean_Co-op,mechanic_clean_Dice_Rolling,mechanic_clean_Movement,mechanic_clean_Other,mechanic_clean_PVP,mechanic_clean_Role_Playing,distance
311,13,Catan,Strategy,Dice_Rolling,7.20,121071,0.375,0.020,0.005333,0.555556,0.479508,0.272727,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.980232e-08
4643,147240,Catan Family Edition,Strategy,Dice_Rolling,6.64,870,0.375,0.020,0.004000,0.555556,0.477459,0.363636,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.094196e-02
2364,38749,Catan Geographies Germany,Strategy,Dice_Rolling,6.89,1129,0.375,0.020,0.002667,0.555556,0.487705,0.363636,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.131681e-02
1500,14,Basari,Strategy,Dice_Rolling,6.78,1716,0.375,0.020,0.001111,0.555556,0.381148,0.181818,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.340041e-01
92,148949,Istanbul,Strategy,Dice_Rolling,7.62,23956,0.250,0.025,0.002667,0.555556,0.530738,0.272727,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.352094e-01
3151,161527,Catan Ancient Egypt,Strategy,Dice_Rolling,7.29,837,0.375,0.020,0.003333,0.555556,0.506148,0.181818,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.460267e-01
2859,153425,North Wind,Strategy,Dice_Rolling,6.45,1641,0.250,0.020,0.002667,0.555556,0.399590,0.272727,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.483880e-01
1007,115,Im the Boss,Strategy,Dice_Rolling,6.81,3652,0.375,0.030,0.002667,0.666667,0.405738,0.363636,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.617385e-01
3068,103091,Catan Histories Merchants of Europe,Strategy,Dice_Rolling,6.76,1343,0.375,0.020,0.005333,0.666667,0.493852,0.181818,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.821032e-01
4213,172994,Catan Traveler Compact Edition,Strategy,Dice_Rolling,7.16,681,0.250,0.020,0.003333,0.555556,0.450820,0.181818,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.925154e-01
